<a href="https://colab.research.google.com/github/Rizwanul-Abidin-Asim/BioHack3D-Authenticating-Biochips-with-Electromagnetic-Fingerprints/blob/main/BioHack3D_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import cv2
import Augmentor
import numpy as np
from rembg import remove
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, auc, roc_curve
from keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications import Xception, InceptionV3, DenseNet121,VGG19,VGG16
from keras.regularizers import l1
import matplotlib.pyplot as plt
from matplotlib import rcParams
from skimage.util import random_noise
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set the path to your dataset directory
dataset_dir1 = r'/content/drive/MyDrive/BioHACK3D 2024_For Participants/Dataset_BioHACK3D_2024/Authentic'
dataset_dir2 = r'/content/drive/MyDrive/BioHACK3D 2024_For Participants/Dataset_BioHACK3D_2024/Inauthentic'
processed_data_dir = r'/content/drive/MyDrive/Processed'

# Set the image dimensions
img_width, img_height = 224, 224

# Load the image filenames and labels
original_files = glob.glob(os.path.join(dataset_dir1, '**','*.jpg'),recursive=True)
print(len(original_files))
duplicate_files = glob.glob(os.path.join(dataset_dir2, '**','*.jpg'),recursive=True)
print(len(duplicate_files))

# Preprocessing Functions
def preprocess_image(image_path):
    with open(image_path, 'rb') as i:
        image_data = i.read()

    # Load the image from the result of rembg
    nparr = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Resize to 224x224
    image = cv2.resize(image, (224, 224))

    # Normalize to [0, 1]
    image = image / 255.0

    return image

# Process and Save Images
def process_and_save_images(input_dir, output_dir, label):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    i=0
    for file_name in input_dir:
        processed_image = preprocess_image(file_name)
        save_path = os.path.join(output_dir, label + "_" + str(i)+".jpg")
        cv2.imwrite(save_path, (processed_image * 255).astype(np.uint8))
        #print(save_path)
        i+=1
process_and_save_images(original_files, processed_data_dir + "/authentic", "authentic")
process_and_save_images(duplicate_files, processed_data_dir + "/non_authentic", "non_authentic")

780
402


In [ ]:
processed_data_dir = r'/content/drive/MyDrive/Processed'  # Adjust the path
augmented_data_dir = r'/content/drive/MyDrive/Augmented'   # Where you want to save augmented images

# Create an output directory for augmented images
if not os.path.exists(augmented_data_dir):
    os.makedirs(augmented_data_dir)

# Create data augmentations for each class
for class_name in ['authentic', 'non_authentic']:
    # Create a pipeline for the current class
    p = Augmentor.Pipeline(os.path.join(processed_data_dir, class_name),
                           output_directory=os.path.join(augmented_data_dir, class_name))

    # Define augmentation operations----you can write code for different augmentation operations
    # Augmentation operations
    p.flip_top_bottom(probability=0.5)
    p.flip_left_right(probability=0.5)  # Adds a left-right flip
    p.rotate(probability=0.7, max_left_rotation=25, max_right_rotation=25)
    p.zoom_random(probability=0.5, percentage_area=0.9)  # Slight zoom, retaining 90% of the original area


    # Sample augmented images (you can adjust this number as needed)
    p.sample(50*100)  # Number of augmented images to create per original image

# Data Generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Make sure to point to the new augmented data directory
train_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
images, labels = next(train_generator)
print("Labels in this batch:", np.sum(labels==0))
plt.figure(figsize=(3, 3 ))
plt.imshow(images[1], cmap='gray')

In [ ]:
#USE THIS CELL
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

# Set up MobileNetV2 base model
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenet_base.trainable = False  # Freeze all layers initially

# Define the model architecture
model = Sequential([
    mobilenet_base,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=l1(0.0001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l1(0.0001)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train with Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    zoom_range=0.1,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    augmented_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train the model with gradual fine-tuning
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs= 20 # Start with fewer epochs
)

# Unfreeze top layers for fine-tuning after initial training
for layer in mobilenet_base.layers[-20:]:  # Adjust number of layers based on performance
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training with unfreezing layers
history_fine_tune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs= 10
)
with open("authenticity_classifier_history.pkl", "wb") as file:
    pickle.dump(history.history, file)

# Save the model in .keras format
model.save('authenticity_classifier.keras')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Function to evaluate precision, recall, F1 score, and confusion matrix
def evaluate_model(model, generator):
    # Predict probabilities
    predictions = model.predict(generator, verbose=1)
    # Convert probabilities to binary predictions
    y_pred = (predictions > 0.5).astype(int).flatten()

    # True labels from the generator
    y_true = generator.classes

    # Calculate metrics
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=generator.class_indices.keys())
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

# Evaluate model on the validation data
evaluate_model(model, validation_generator)
